# ¡Bienvenido a Ingeniería de Características!

En este curso aprenderás sobre uno de los pasos más importantes para construir un gran modelo de *machine learning*: la **ingeniería de características**. Aprenderás a:

- Determinar qué características son las más importantes usando **información mutua**.
- Inventar nuevas características en varios dominios de problemas del mundo real.
- Codificar variables categóricas de **alta cardinalidad** mediante **target encoding**.
- Crear características de **segmentación** con **clustering k-means**.
- Descomponer la variación de un conjunto de datos en características usando **análisis de componentes principales (PCA)**.

Los ejercicios prácticos culminan en un *notebook* completo que aplica todas estas técnicas para realizar una entrega a la competencia **House Prices Getting Started**. Después de completar este curso, tendrás varias ideas que podrás usar para mejorar aún más tu rendimiento.

¿Estás listo? ¡Vamos!








# El objetivo de la ingeniería de características

El objetivo de la ingeniería de características es, simplemente, hacer que tus datos estén mejor adaptados al problema que tienes entre manos.

Considera medidas de **“temperatura aparente”** como el índice de calor (*heat index*) y la sensación térmica por viento (*wind chill*). Estas magnitudes intentan medir la temperatura percibida por los seres humanos basándose en la temperatura del aire, la humedad y la velocidad del viento, variables que podemos medir directamente. Puedes pensar en la temperatura aparente como el resultado de una especie de ingeniería de características: un intento de hacer que los datos observados sean más relevantes para lo que realmente nos importa, ¡cómo se siente en realidad estar afuera!

Podrías realizar ingeniería de características para:

- Mejorar el rendimiento predictivo de un modelo.
- Reducir las necesidades computacionales o de datos.
- Mejorar la interpretabilidad de los resultados.







## Un principio guía de la ingeniería de características

Para que una característica sea útil, debe tener una relación con el objetivo (*target*) que tu modelo sea capaz de aprender. Los modelos lineales, por ejemplo, solo pueden aprender relaciones lineales. Por lo tanto, cuando se utiliza un modelo lineal, tu objetivo es transformar las características de manera que su relación con el objetivo sea lineal.

La idea clave aquí es que una transformación que aplicas a una característica se convierte, en esencia, en parte del propio modelo. Supongamos que intentas predecir el **precio** de terrenos cuadrados a partir de la **longitud** de uno de sus lados. Ajustar un modelo lineal directamente sobre la longitud produce malos resultados: la relación no es lineal.


Si elevamos al cuadrado la característica **Length** para obtener **Area**, sin embargo, creamos una relación lineal. Al añadir **Area** al conjunto de características, este modelo lineal ahora puede ajustar una parábola. En otras palabras, elevar una característica al cuadrado le dio al modelo lineal la capacidad de ajustarse a características cuadráticas.


Esto debería mostrarte por qué la ingeniería de características puede ofrecer un retorno tan alto sobre el tiempo invertido. Cualquier relación que tu modelo no pueda aprender, puedes proporcionársela tú mismo mediante transformaciones. A medida que desarrolles tu conjunto de características, piensa en qué información podría utilizar tu modelo para alcanzar su mejor rendimiento.


---

<p align="center">
  <img src="assets/separador.png" alt="Separador" width=1000"/>
</p>

---

## Ejemplo – Formulaciones de concreto

Para ilustrar estas ideas, veremos cómo añadir algunas características sintéticas a un conjunto de datos puede mejorar el rendimiento predictivo de un modelo de **random forest**.

El conjunto de datos **Concrete** contiene una variedad de formulaciones de concreto y la **resistencia a la compresión** del producto resultante, que es una medida de cuánta carga puede soportar ese tipo de concreto. La tarea de este conjunto de datos es predecir la resistencia a la compresión de un concreto dada su formulación.


In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [5]:
df=pd.read_csv("data/concrete.csv")

In [7]:
df.head()


,Cement,BlastFurnaceSlag,FlyAsh,Water,Superplasticizer,CoarseAggregate,FineAggregate,Age,CompressiveStrength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Aquí puedes ver los distintos ingredientes que intervienen en cada variedad de **hormigón**. En un momento veremos cómo añadir algunas características sintéticas adicionales derivadas de estos ingredientes puede ayudar a un modelo a aprender relaciones importantes entre ellos.

Primero estableceremos una **línea base** entrenando el modelo con el conjunto de datos sin aumentar (sin características adicionales). Esto nos ayudará a determinar si nuestras nuevas características son realmente útiles.

Establecer líneas base como esta es una buena práctica al inicio del proceso de ingeniería de características. Una puntuación de referencia puede ayudarte a decidir si vale la pena conservar tus nuevas características o si deberías descartarlas y, posiblemente, probar algo diferente.


In [11]:


X=df.copy()

# Copiamos en X el df con .copy(). X contendrá todas las variables, incluida la variable objetivo(por ahora)

# Buena practica para evitar efectos secundarios

y=X.pop("CompressiveStrength")

# con .pop() extraemos la columna "CompressiveStrength, la eliminamos de X y la devuelve para ser mas precisos. X ahora queda solo con las features.

# X -> ingredientes del hormigón

# y -> resistencia a la compresión del hormigón

baseline=RandomForestRegressor(criterion="absolute_error",random_state=0)

# Creamos, definimos el modelo(baseline) con un RandomForestRegressor

# Criterion="absolute_error". El modelo optimiza el error absoluto medio (MAE) en lugar del error cuadrático

# random_state=0 hace que los resultados sean reproducibles

# Este modelo sirve como punto de referencia antes de hacer "feature Engineering"

baseline_score=cross_val_score(
    baseline,
    X,
    y,
    cv=5,
    scoring="neg_mean_absolute_error")

# Este trozo evalua el modelo usando validacion cruzada

# cv=5 divide los datos en 5 folds. Entrena con 4 y valida con 1, repitiendo 5 veces

# scoring="neg_mean_absolute_error" devuelve el MAE como valor negativo. Caunto mas cerca de 0, mejor.

# baseline_Score es un array de 5 valores(uno por fold)

baseline_score = -1 * baseline_score.mean()

# se multiplica por -1 para convertir MAE a un valor positivo

# .mean() calcula el promedio de los 5 folds

# Esto nos da un MAE promedio, mas estable y confiable

print(f"MAE Baseline Score: {baseline_score:.4}")


MAE Baseline Score: 8.326


Si alguna vez cocinas en casa, quizá sepas que la **proporción** entre los ingredientes de una receta suele ser un mejor predictor del resultado final que sus cantidades absolutas. Siguiendo este razonamiento, podríamos pensar que las **razones (ratios)** entre las características anteriores serían un buen predictor de la **resistencia a la compresión**.

La celda siguiente añade **tres nuevas características basadas en ratios** al conjunto de datos.


In [12]:
X = df.copy()
y = X.pop("CompressiveStrength")

# Creamos features sintéticas
X["FCRatio"] = X["FineAggregate"] / X["CoarseAggregate"]
X["AggCmtRatio"] = (X["CoarseAggregate"] + X["FineAggregate"]) / X["Cement"]
X["WtrCmtRatio"] = X["Water"] / X["Cement"]


# Entrenamos y obtenemos el score con las nyevas features sinteticas
model = RandomForestRegressor(criterion="absolute_error", random_state=0)
score = cross_val_score(
    model, X, y, cv=5, scoring="neg_mean_absolute_error"
)
score = -1 * score.mean()

print(f"MAE Score with Ratio Features: {score:.4}")

MAE Score with Ratio Features: 7.951


Y, efectivamente, ¡el rendimiento mejoró! Esto es una evidencia de que estas nuevas características basadas en **ratios** expusieron información importante al modelo que antes no estaba siendo detectada.


---

<p align="center">
  <img src="assets/separador.png" alt="Separador" width=1000"/>
</p>

---